In [5]:
import tensorflow as tf
import numpy as np

# 导入TF相关模块
from tensorflow.keras import layers
from tensorflow import keras

# 11. 循环神经网络
卷积神经网络利用数据的局部相关性和权值共享的思想大大减少了网络的参数量，非常适合于图片这种具有空间(Spatial)局部相关性的数据。

自然界的信号除了具有空间维度之外，还有一个时间(Temporal)维度，比如文本、语音信号、股市参数等。这类数据并不一定具有局部相关性，同时数据在时间维度上的长度也是可变的，卷积神经网络并不擅长处理此类数据。

本章介绍的循环神经网络可以较好地解决此类问题。

## 11.1 序列表示方法
具有先后顺序的数据一般叫作`序列`(Sequence)，比如随时间而变化的商品价格数据。考虑某件商品A在1月到6月之间的价格变化趋势，我们记为一维向 量：$[x_1, x_2, x_3, x_4, x_5, x_6]$，它的`shape` 为$[6]$。如果要表示$b$件商品在1月到6月之间的价格变化趋势，可以记为2维张量：
+ $\bigg[\big[x_1^{(1)}, x_2^{(1)}, x_3^{(1)}, x_4^{(1)}, x_5^{(1)}, x_6^{(1)}\big], \big[x_1^{(2)}, x_2^{(2)}, x_3^{(2)}, x_4^{(2)}, x_5^{(2)}, x_6^{(2)}\big],\dots,\big[x_1^{(b)}, x_2^{(b)}, x_3^{(b)}, x_4^{(b)}, x_5^{(b)}, x_6^{(b)}\big]\bigg]$

其中$b$表示商品的数量，张量`shape`为$[b, 6]$。

这么看来，序列信号表示起来并不麻烦，只需要一个`shape`为$[b, s]$的张量即可，其中$b$为序列数量，$s$为序列长度。但是对于很多信号并不能直接用一个标量数值表示，比如每个时间戳产生长度为$n$的特征向量，则需要`shape`为$[b, s, n]$的张量才能表示。

考虑更复杂的文本数据：句子。它在每个时间戳上面产生的单词是一个字符，并不是数值，不能直接用某个标量表示。神经网络不能够直接处理字符串类型的数据，需要把单词用`One-hot`编码。

我们把文字编码为数值的过程叫作`Word Embedding`。`One-hot`编码是最简单的`Word Embedding`实现。但是`One-hot`编码的向量是高维度而 且极其稀疏的，计算效率较低，也不利于神经网络的训练。从语义角度来讲，`One-hot`编码忽略了单词先天具有的语义相关性。举个例子，对于单词`like`、`dislike`、`Rome`、`Paris`来说，`like`和`dislike`在语义角度就强相关，它们都表示喜欢的程度；`Rome`和`Paris`同样也是强相关，他们都表示欧洲的两个地点。如果采用`One-hot`编码，得到的向量之间没有相关性，不能很好地体现原有文字的语义相关度。

在自然语言处理领域，有专门的一个研究方向在探索如何学习到`单词表示向量`(Word Vector)，使得语义层面的相关性能够很好地通过`Word Vector`体现出来。一个衡量词向量之间相关度的方法就是`余弦相关度`(Cosine similarity)：
+ $\text{similarity}(a,b) \triangleq \cos(\theta) = \frac{a\cdot b}{\lvert a\rvert \cdot \lvert b\rvert}$

其中$a$和$b$代表了两个词向量。`图11.2`演示了单词`France`和`Italy`的相似度，以及单词`ball`和`crocodile`的相似度，可以看到$\cos(\theta)$较好地反映了语义相关性：

<img src="images/11_02.png" style="width:400px;"/>

### 11.1.1 Embedding层
在神经网络中，单词的表示向量可以直接通过训练的方式得到，我们把单词的表示层叫作`Embedding`层。`Embedding`层负责把单词编码为某个词向量。

`Embedding`层是可训练的，它可放置在神经网络之前，完成单词到向量的转换，得到的表示向量可以继续通过神经网络完成后续任务，并计算误差$\mathcal{L}$，采用梯度下降算法来实现端到端(end-to-end)的训练。

`TensorFlow`通过`layers.Embedding()`来定义一个`Word Embedding`层：

In [8]:
# 生成10个单词的数字编码
x = tf.range(10)
# 打散
x = tf.random.shuffle(x) 
# 创建共10个单词，每个单词用长度为4的向量表示的层
net = layers.Embedding(10, 4)
# 获取词向量
out = net(x) 

上述代码创建了10个单词的`Embedding`层，每个单词用长度为4的向量表示，可以传入数字编码为`0~9`的输入，得到这4个单词的词向量，这些词向量随机初始化的，尚未经过网络训练：

In [9]:
out

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[ 0.01157155, -0.00832675,  0.02514343, -0.03386571],
       [-0.0276983 , -0.01975003,  0.02297426,  0.00017058],
       [ 0.00254769,  0.01155359,  0.02799598,  0.01360115],
       [-0.0297976 , -0.03740866,  0.02471049,  0.03253405],
       [ 0.00397237, -0.01465958,  0.01788909, -0.03889204],
       [-0.01133395,  0.03655943,  0.02940017, -0.01093923],
       [ 0.0462333 , -0.03355135,  0.04081309, -0.04069991],
       [-0.0001731 , -0.01660696,  0.04790676,  0.01318611],
       [ 0.03445579, -0.02370908, -0.01526465, -0.00193327],
       [ 0.01685118,  0.03224106,  0.0042236 ,  0.01313618]],
      dtype=float32)>

我们可以直接查看`Embedding`层内部的查询表`table`。可以看到`net.embeddings`张量的可优化属性为`True`，即可以通过梯度下降算法优化。

In [10]:
net.embeddings

<tf.Variable 'embedding_2/embeddings:0' shape=(10, 4) dtype=float32, numpy=
array([[ 0.01157155, -0.00832675,  0.02514343, -0.03386571],
       [ 0.00397237, -0.01465958,  0.01788909, -0.03889204],
       [ 0.0462333 , -0.03355135,  0.04081309, -0.04069991],
       [-0.0276983 , -0.01975003,  0.02297426,  0.00017058],
       [-0.0297976 , -0.03740866,  0.02471049,  0.03253405],
       [ 0.01685118,  0.03224106,  0.0042236 ,  0.01313618],
       [ 0.00254769,  0.01155359,  0.02799598,  0.01360115],
       [ 0.03445579, -0.02370908, -0.01526465, -0.00193327],
       [-0.01133395,  0.03655943,  0.02940017, -0.01093923],
       [-0.0001731 , -0.01660696,  0.04790676,  0.01318611]],
      dtype=float32)>

### 11.1.2 预训练的词向量
`Embedding`层的查询表是随机初始化的，需要从零开始训练。实际上，我们可以使用预训练的`Word Embedding`模型来得到单词的表示方法，基于预训练模型的词向量相当于迁移了整个语义空间的知识，往往能得到更好的性能。

目前应用的比较广泛的预训练模型有`Word2Vec`和`GloVe`等，它们已经在海量语料库训练得到了较好的词向量表示方法。比如`GloVe`模型`GloVe.6B.50d`，词汇量为40万，每个单词使用长度为50的向量表示。

那么如何使用这些预训练的词向量模型来帮助提升`NLP`任务的性能？非常简单，对于`Embedding`层，不再采用随机初始化的方式，而是利用我们已经预训练好的模型参数去初始化`Embedding`层的查询表：

```python
# 从预训练模型中加载词向量表
embed_glove = load_embed('glove.6B.50d.txt')
# 直接利用预训练的词向量表初始化 Embedding 层
net.set_weights([embed_glove])
```

经过预训练的词向量模型初始化的`Embedding`层可以设置为不参与训练：`net.trainable = False`，那么预训练的词向量就直接应用到此特定任务上；如果希望能够学到区别于预训练词向量模型不同的表示方法，那么可以把`Embedding`层包含进反向传播算法中去，利用梯度下降来微调单词表示方法。

## 11.2 循环神经网络
以文本序列为例：
> “I hate this boring movie”

通过`Embedding`层，可以将它转换为`shape`为$[b,s,n]$的张量，$b$为句子数量，$s$为句子长度，$n$为词向量长度。上述句子可以表示为`shape`为$[1,5,10]$的张量。

我们以情感分类任务为例逐步探索能够处理序列信号的网络模型，如`图11.3`所示。

<img src="images/11_03.png" style="width:400px;"/>

情感分类任务通过分析给出的文本序列，提炼出文本数据表达的整 体语义特征，从而预测输入文本的情感类型：正面评价或者负面评价。从分类角度来看， 情感分类问题就是一个简单的二分类问题，与图片分类不一样的是，由于输入是文本序 列，传统的卷积神经网络并不能取得很好的效果。那么什么类型的网络擅长处理序列数据 呢？

主训练函数实现如下：

In [6]:
def main():
    lr = 0.01 # 学习率
    initial_b = 0 # 初始化b为0
    initial_w = 0 # 初始化w为0
    num_iterations = 1000
    # 训练优化 1000 次，返回最优 w*,b*和训练 Loss 的下降过程
    [b, w]= gradient_descent(data, initial_b, initial_w, lr, num_iterations)
    loss = mse(b, w, data) # 计算最优数值解 w,b 上的均方差
    print(f'Final loss:{loss}, w:{w}, b:{b}')

main()

iteration:0, loss:5.5559108101397365, w:1.0878737485740488, b:-0.01088794329648056
iteration:100, loss:0.00029992466589305237, w:1.4767873581796336, b:0.07466228872146469
iteration:200, loss:0.00010959378778883515, w:1.4769318134316611, b:0.08674836727243838
iteration:300, loss:0.00010617553247045912, w:1.4769511723442907, b:0.08836806150590652
iteration:400, loss:0.00010611414217195506, w:1.4769537666943953, b:0.08858512193612501
iteration:500, loss:0.00010611303963033697, w:1.4769541143715914, b:0.08861421090205028
iteration:600, loss:0.00010611301982919316, w:1.47695416096493, b:0.08861810920787117
iteration:700, loss:0.00010611301947357429, w:1.4769541672090531, b:0.08861863163236407
iteration:800, loss:0.00010611301946718586, w:1.476954168045848, b:0.08861870164414955
iteration:900, loss:0.00010611301946707576, w:1.4769541681579896, b:0.08861871102665395
Final loss:0.00010611301946706803, w:1.476954168172971, b:0.08861871228008317


上述例子比较好地展示了梯度下降算法在求解模型参数上的强大之处。

需要注意的是，对于复杂的非线性模型，通过梯度下降算法求解到的$w$和$b$可能是局部极小值而非全局最小值解，这是由模型函数的非凸性决定的。但是我们在实践中发现，通过梯度下降算法求得的数值解，它的性能往往都能优化得很好，可以直接使用求解到的数值解$w$和$b$来近似作为最优解。